# Import libraries

In [36]:
from math import sqrt
from numpy import array
from numpy import mean
import pandas as pd
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from pandas.io.json import json_normalize
from sklearn.metrics import explained_variance_score,max_error,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten
from tensorflow.python.keras.layers.convolutional import Conv1D
from tensorflow.python.keras.layers.convolutional import MaxPooling1D
from datetime import datetime

In [37]:
def train_test_split(data, n_test):
    return data[:int(0.8*data.shape[0])], data[int(0.8*data.shape[0]):]

## Perceptron

In [38]:
def perceptron_model(config):
    *_, n_features, n_input= config
    model_mlp = Sequential()
    model_mlp.add(Dense(units=50, input_shape=(n_input, n_features)))
    model_mlp.add(Dense(1))
    model_mlp.compile(optimizer="adam", loss="mean_squared_error")
    return model_mlp

## LSTM 

In [39]:
def lstm_model(config):
    *_, n_features, n_input, n_nodes = config
    model = Sequential()
    model.add(LSTM(n_nodes, activation='relu', input_shape=(n_input, n_features)))
    model.add(Dense(n_nodes, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
#     model.name="LSTM"
    return model

## Conv1D

In [40]:
def conv1d_model(config):
    *_, n_features, n_input, n_filters, n_kernel = config
    model = Sequential()
    model.add(Conv1D(filters=n_filters, kernel_size=n_kernel, activation='relu', input_shape=(n_input, n_features)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
#     model.name="Conv1D"
    return model

In [41]:
def configuration(name, *args):
    name = name.split("-")[0]
    if(name=="perceptron"):
        n_input = args[0]
        n_nodes = args[1]
        n_epochs = args[2]
        n_batch = args[3]
        n_diff = args[4]
        n_features = args[5]
        configs = list()
        for i in n_epochs:
            for j in n_batch:
                for k in n_diff:
                    for l in n_features:
                        for m in n_input:
                            for n in n_nodes:
                                cfg = [i, j, k, l, m, n]
                                configs.append(cfg)
        return (configs, ["n_epochs","n_batch","n_diff","n_features","n_input","n_nodes"])
    
    if(name=="lstm"):
        n_input = args[0]
        n_nodes = args[1]
        n_epochs = args[2]
        n_batch = args[3]
        n_diff = args[4]
        n_features = args[5]
        configs = list()
        for i in n_epochs:
            for j in n_batch:
                for k in n_diff:
                    for l in n_features:
                        for m in n_input:
                            for n in n_nodes:
                                cfg = [i, j, k, l, m, n]
                                configs.append(cfg)
        return (configs, ["n_epochs","n_batch","n_diff","n_features","n_input","n_nodes"])
    
    if(name=="conv1d"):
        n_input = [12]
        n_filters = [64]
        n_kernel = [3]
        n_epochs = [50]
        n_batch = [1]
        n_diff = [12]
        n_features = [1]
        configs = list()
        for i in n_epochs:
            for j in n_batch:
                for k in n_diff:
                    for l in n_features:
                        for m in n_input:
                            for n in n_filters:
                                for o in n_kernel:
                                    cfg = [i, j, k, l, m, n, o]
                                    configs.append(cfg)
        return (configs, ["n_epochs","n_batch","n_diff","n_features","n_input","n_filters","n_kernel" ])

In [42]:
series = read_csv('airline-passengers.csv', header=0, index_col=0)
data = series.values
n_test = 12

In [43]:
# transform list into supervised learning format
def series_to_supervised(data, n_in=1, n_out=1):
    df = DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = concat(cols, axis=1)
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg.values

In [44]:
def measure_error(actual, predicted):
    return [explained_variance_score(actual, predicted),max_error(actual, predicted),
            mean_absolute_error(actual, predicted),mean_squared_error(actual, predicted),
            sqrt(mean_squared_error(actual, predicted)),mean_squared_log_error(actual, predicted),
            median_absolute_error(actual, predicted),r2_score(actual, predicted)]
                                                    

In [45]:
def difference(data, order):
    return [data[i] - data[i - order] for i in range(order, len(data))]

In [46]:
def model_fit(train, config, create_model):
    # unpack config
    n_epochs, n_batch, n_diff,n_features,n_input,*_ = config
    # prepare data
    if n_diff > 0:
        train = difference(train, n_diff)
    # transform series into supervised format
    data = series_to_supervised(train, n_in=n_input)
    # separate inputs and outputs
    train_x, train_y = data[:, :-1], data[:, -1]
    # reshape input data into [samples, timesteps, features]
    train_x = train_x.reshape((train_x.shape[0], train_x.shape[1], n_features))
    # define model
#     model = lstm_model(config);
    model = create_model
    # fit
    model.fit(train_x, train_y, epochs=n_epochs, batch_size=n_batch, verbose=0)
    return model

In [47]:
# forecast with the fit model
def model_predict(model, history, config, create_model):
    # unpack config
    _,_,n_diff,_,n_input,*_ = config
    # prepare data
    correction = 0.0
    if n_diff > 0:
        correction = history[-n_diff]
        history = difference(history, n_diff)
    x_input = array(history[-n_input:]).reshape((1, n_input, 1))
    # forecast
    yhat = model.predict(x_input, verbose=0)
    return correction + yhat[0]

In [48]:
def walk_forward_validation(data, n_test, cfg, create_model,array_of_results):
    predictions = list()
    # split dataset
    train, test = train_test_split(data, n_test)
    # fit model
    model = model_fit(train, cfg, create_model)
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # fit model and make forecast for history
        yhat = model_predict(model, history, cfg, create_model)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
    # estimate prediction error
    error = measure_error(test, predictions)

    return error

In [49]:
def repeat_evaluate(data, config, n_test,create_model,array_of_results,keys, n_repeats=2):
    #MOŻE TU BYĆ PROBLEM W JAK ZMIENIMY n_repeats
    scores = [walk_forward_validation(data, n_test, config, create_model,array_of_results) for _ in range(n_repeats)]
    results ={}
    scores = zip(scores[0],scores[1])
    result = []
    for s in scores:
        result.append(mean(s))
        
    if "Model" not in results:
        results["Model"] = list()
    results["Model"].extend([create_model.name])

#     for k,c in zip(keys,config):
#         if str(k) not in results:
#             results[str(k)] = list()
#         results[str(k)].extend([c])

    if "EVC" not in results:
        results["EVC"] = list()
    results["EVC"].extend([result[0]])

    if "ME" not in results:
        results["ME"] = list()
    results["ME"].extend([result[1]])

    if "MAE" not in results:
        results["MAE"] = list()
    results["MAE"].extend([result[2]])

    if "MSE" not in results:
        results["MSE"] = list()
    results["MSE"].extend([result[3]])

    if "RMSE" not in results:
        results["RMSE"] = list()
    results["RMSE"].extend([result[4]])

    if "MSLE" not in results:
        results["MSLE"] = list()
    results["MSLE"].extend([result[5]])

    if "MED" not in results:
        results["MED"] = list()
    results["MED"].extend([result[6]])

    if "R2" not in results:
        results["R2"] = list()
    results["R2"].extend([result[7]])

  
    return results

In [50]:
def update_model(name,*args):
    return configuration(name, *args)

In [56]:
def run():

    cfg_list = []
    cfg_list.append(lstm_model)
    cfg_list.append(conv1d_model)
#     cfg_list.append(perceptron_model)
    
    default_config = []
    default_config.append(([12],[100],[2,3],[1],[12],[1]))
    default_config.append(([12],[64],[3],[50],[1],[12],[1]))
#     default_config.append(([12],[100],[50],[1],[12],[1]))
    array_of_results = []
    results = {}
    array_of_models = []
    for cfg_model,def_cfg in zip(cfg_list,default_config):
        i=1
        results.clear()
#         config = configuration(str(cfg_model.__name__.split("_")[0]), )[0]
        config = update_model(str(cfg_model.__name__.split("_")[0]),(*def_cfg))[0]
        for cfg in config:
            model = cfg_model(cfg)
            model._name=str(cfg_model.__name__.split("_")[0]+"-"+str(i))
            i +=1
            array_of_models.append(model)
            array_of_results.append(repeat_evaluate(data, cfg, n_test, model, array_of_results,configuration(str(cfg_model.__name__.split("_")[0]),(*def_cfg))[1]))
                    
    array_of_dataframes = []
    for i in array_of_results:
        array_of_dataframes.append(pd.DataFrame.from_dict(i))
        
    return (array_of_dataframes, array_of_models)
    

In [61]:
def run_model(model_name, conf, result_df):
    array_of_results = []
    cfg = update_model(str(model_name),(*conf))[0][0]
    i = int(model_name.split("-")[1])
    model = eval(model_name.split("-")[0]+"_model")(cfg)
    model._name=str(model_name.split("-")[0]+"-"+str(i+1))
    result = repeat_evaluate(data, cfg, n_test, model, array_of_results,configuration(str(model.name),cfg)[1])
    result = pd.DataFrame.from_dict(result)
    return pd.concat([result,result_df]), model

In [58]:
def find_max(df_array,stat):
    max =0
    model = ""
    for i in df_array:
        if(max <= int(i.iloc[0][stat])):
            model = i.iloc[0]["Model"]
    return model

In [59]:
def get_model(list_of_models,model):
    index = -1;
    for m in list_of_models:
        index += 1
        if(model==m.name):
            return (m,index)

In [60]:
dfs,models = run()

In [62]:
def test():  
    df_model = dfs[get_model(models,find_max(dfs,"R2"))[1]]
    df_model.head()
    df_model, model = run_model(df_model.iloc[0]["Model"],([0],[12],[55],[3],[50],[1],[12],[1]),df_model)
    models.append(model)
    df_model, model = run_model(df_model.iloc[0]["Model"],([0],[12],[60],[3],[50],[1],[12],[1]),df_model)
    models.append(model)
    return df_model


In [ ]:
test().head()

In [23]:
def get_config(list_of_models):
    list_of_configs = {}
    for m in list_of_models:
        list_of_configs[m.get_config()['name']] = m.get_config()['layers']
    return list_of_configs

In [29]:
def convert_config_to_datframes(config_json):

    index = -1
    tmp = []
    list_of_final_dataframes = []
    dfJson1 =None
    for i in config_json:
        if dfJson1 is not None:
            list_of_final_dataframes.append(dfJson1)
        dfJson1 = pd.DataFrame(index=[],columns=['name'])
        for j in config_json[i]:       
            index += 1
            dfJson1.loc[index,'date']=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            dfJson1.loc[index,'class_name']=j['class_name']
            dfJson1.loc[index,'name']=i
            for k in j['config']:
                dfJson1.loc[index,'cfg-'+str(k)]=str(j['config'][k]) 
    return list_of_final_dataframes

In [30]:
convert_config_to_datframes(get_config(models))[0].head(25)

,name,date,class_name,cfg-name,cfg-trainable,cfg-batch_input_shape,cfg-dtype,cfg-return_sequences,cfg-return_state,cfg-go_backwards,...,cfg-kernel_regularizer,cfg-recurrent_regularizer,cfg-bias_regularizer,cfg-activity_regularizer,cfg-kernel_constraint,cfg-recurrent_constraint,cfg-bias_constraint,cfg-dropout,cfg-recurrent_dropout,cfg-implementation
0,lstm-1,2020-11-01 21:32:16,LSTM,lstm_1,True,"(None, 12, 1)",float32,False,False,False,...,None,None,None,None,None,None,None,0.0,0.0,2
1,lstm-1,2020-11-01 21:32:16,Dense,dense_1,True,NaN,float32,NaN,NaN,NaN,...,None,NaN,None,None,None,NaN,None,NaN,NaN,NaN
2,lstm-1,2020-11-01 21:32:16,Dense,dense_2,True,NaN,float32,NaN,NaN,NaN,...,None,NaN,None,None,None,NaN,None,NaN,NaN,NaN
